In [1]:
import functools
import operator
import requests
import os
import json

from bs4 import BeautifulSoup

from duckduckgo_search import DDGS
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import HumanMessage, BaseMessage
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI


from typing import TypedDict, Annotated, Sequence

from langchain_community.tools.tavily_search import TavilySearchResults
from decouple import config

from dotenv import load_dotenv
from pathlib import Path
os.chdir("../../")
load_dotenv('.env', override=True)
print(Path.cwd())

/home/mymm_psu_gmail_com/hackathon/rag-agents/multimodal-conv-agents


/home/mymm_psu_gmail_com/miniconda3/envs/dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set environment variables
assert os.environ["OPENAI_API_KEY"]
assert os.environ["TAVILY_API_KEY"]

# Initialize model
llm = ChatOpenAI(model="gpt-3.5-turbo")
# or portkey
from agents.utils.portkey import get_portkey_langchain_llm
# not supporting bind tools
# llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

In [3]:
# define custom tools to use
@tool("process_search_tool", return_direct=False)
def process_search_tool(url: str) -> str:
    """Used to process content found on the internet."""
    response = requests.get(url=url)
    soup = BeautifulSoup(response.content, "html.parser")
    return soup.get_text()

@tool("internet_search_tool", return_direct=False)
def internet_search_tool(query: str, max_result=3) -> str:
    """Search provided query on the internet using DuckDuckGo"""
    with DDGS() as ddgs:
        results = [r for r in ddgs.text(query, max_results=max_result)]
        return results if results else "No results found"


tools = [TavilySearchResults(max_results=1), process_search_tool]

In [4]:
def create_agents(llm: ChatOpenAI,
                  tools: list,
                  system_prompt: str) -> AgentExecutor:
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ])

    # crate agent with tools
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor


def agent_node(state, agent, name):
    result = agent.invoke(state)
    
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [5]:
# List of agents
members = ["news_correspondent", "news_editor", "ads_writter"]


system_prompt = (
    "As a supervisor, your role is to oversee the insight between these"
    " workers: {members}. Based on the user's request,"
    " determine which worker should take the next action. Each worker is responsible for"
    " executing a specific task and reporting back thier findings and progress."
    " Once all tasks are completed, indicate 'FINISH'."
)

options = ["FINISH"] + members

function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {"next": {"title": "Next", "anyOf": [{"enum": options}]}},
        "required": ["next"]
    }
}


prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="messages"),
    ("system",
     "Given the conversation above, who should act next? Or should we FINISH? Select one of: {options}"),
]).partial(options=str(options), members=", ".join(members))

supervisor_chain = (
    prompt | llm.bind_functions(
        functions=[function_def], function_call="route"
    ) | JsonOutputFunctionsParser()
)

In [9]:
news_correspondent_agent = create_agents(
    llm,
    tools,
    """Your primary role is to function as an intelligent news research assistant, adept at scouring 
    the internet for the latest and most relevant trending stories across various sectors like politics, technology, 
    health, culture, and global events. You possess the capability to access a wide range of online news sources, 
    blogs, and social media platforms to gather real-time information."""
)

news_editor_agent = create_agents(
    llm, tools,
    """You are a news editor. Do step by step approach. 
        Based on the provided content first identify the list of topics,
        then search internet for each topic one by one
        and finally find insights for each topic one by one that can aid you 
        in writting a useful news edition for AI-nes corp.
        Include the insights and sources in the final response
        """)

ads_writter_agent = create_agents(
    llm, tools,
    """You are an ads writter for AI-news corp. Given the publication generated by the
    news editor, your work if to write ads that relate to that content. Use the internet 
    to search for content to write ads based off on. Here is a description of your task:
    
    To craft compelling and relevant advertisements for 'AI News' publication, complementing the content written by the news editor.
    Contextual Ad Placement: Analyze the final report content from the news editor in-depth to identify key themes, topics, 
    and reader interests. Place ads that are contextually relevant to these findings, thereby increasing potential customer engagement.
    Advanced Image Sourcing and Curation: Employ sophisticated web search algorithms to source high-quality, relevant images for each ad. 
    Ensure these images complement the ad content and are aligned with the publication's aesthetic standards.
    Ad-Content Synchronization: Seamlessly integrate advertisements with the report, ensuring they enhance rather than disrupt the reader's 
    experience. Ads should feel like a natural extension of the report, offering value to the reader.
    Reference and Attribution Management: For each image sourced, automatically generate and include appropriate references and attributions, 
    ensuring compliance with copyright laws and ethical standards.
    """)

In [10]:

news_correspondent_node = functools.partial(
    agent_node, agent=news_correspondent_agent, name="news_correspondent"
)

# changed from news_editor_node => news_editor
news_editor_node = functools.partial(
    agent_node, agent=news_editor_agent, name="news_editor")


ads_writter_node = functools.partial(
    agent_node, agent=ads_writter_agent, name="ads_writter")

In [11]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str


# Create workflow or graph
workflow = StateGraph(AgentState)

# adding nodes
workflow.add_node("supervisor", action=supervisor_chain)
workflow.add_node("news_correspondent", action=news_correspondent_node)
workflow.add_node("news_editor", action=news_editor_node)
workflow.add_node("ads_writter", action=ads_writter_node)

In [12]:
# define edgs
for member in members:
    workflow.add_edge(start_key=member, end_key="supervisor")


conditional_map = {k: k for k in members}
conditional_map['FINISH'] = END

# if task is FINISHED, supervisor won't send task to agent, else,
# the supervisor will keep on sending task to agent untill done, this is
# what the conditional edge does.
workflow.add_conditional_edges(
    "supervisor", lambda x: x["next"], conditional_map)
workflow.set_entry_point("supervisor")

graph = workflow.compile()

In [14]:
print(f"Branches:")
print(workflow.branches)
print(f"\nEdges:")
print(workflow.edges)
print(f"\nnodes:")
print(workflow.nodes)
print(f"\nchannels:")
print(workflow.channels)

Branches:
defaultdict(<class 'dict'>, {'supervisor': {'condition': Branch(path=RunnableCallable(recurse=True), ends={'news_correspondent': 'news_correspondent', 'news_editor': 'news_editor', 'ads_writter': 'ads_writter', 'FINISH': '__end__'}, then=None)}})

Edges:
{('news_editor', 'supervisor'), ('news_correspondent', 'supervisor'), ('ads_writter', 'supervisor'), ('__start__', 'supervisor')}

nodes:
{'supervisor': ChatPromptTemplate(input_variables=['messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'options': "['FINISH', 'news_correspondent', 'news_editor', 'ads_writter']", 'members': 'news_correspondent, news_editor, ads_writter'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(in

In [15]:
message_stream = []
verbose = False
# 1. Stream
for s in graph.stream(
    {
        "messages": [
            HumanMessage(
                content="""
                    Write me a report on the Obama Administration and a summary of the impact. \nAfter the research on historic events during the obama administration, pass a summary of findings under 1000 words to the news editor to generate the final publication.
                    Once done, pass it to the ads writter to write the ads on the subject in less than 2000 words in markdown text with approrpiate format for readability and engagement.
                """
            )
        ],
    },
    
    # Maximum number of steps to take in the graph
    {"recursion_limit": 150}
):
    message_stream.append(s)
    if not "__end__" in s:
        if verbose:
            print(s, end="\n\n-----------------\n\n")

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: X. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
message_stream

[{'supervisor': {'next': 'news_correspondent'}},
 {'news_correspondent': {'messages': [HumanMessage(content="The research on historic events during the Obama Administration reveals significant milestones and impactful decisions made by President Barack Obama from 2009 to 2017. Here are some key events highlighted in the timeline:\n\n- January 20, 2009: Inaugural Address and Executive Orders Ensuring Lawful Interrogations and Review of Guantanamo Bay Detention Center.\n- January 29, 2009: Signing of the Lilly Ledbetter Fair Pay Act of 2009.\n- February 17, 2009: Signing of the American Recovery and Reinvestment Act of 2009.\n- March 23, 2010: Signing of the Patient Protection and Affordable Care Act.\n- April 20, 2010: Deepwater Horizon Oil Spill.\n- January 16, 2013: Remarks on Gun Violence.\n- July 21, 2014: Signing of Executive Order on Lesbian, Gay, Bisexual, and Transgender Employment Discrimination.\n- June 26, 2015: U.S. Supreme Court ruling on same-sex marriage.\n- January 21, 2

In [ ]:
# 2. No Streaming
# final_respone = graph.invoke({
#     "messages": [HumanMessage(content="""Write me a report on spaceX. After the research on spaceX,
#                               pass the findings to the news editor to generate the final publication.
#                               Once done, pass it to the ads writter to write the ads on the subject.""")]
# }, {"recursion_limit": 150})

# print(final_respone["messages"][1].content)


# def run_graph(input_message):
#     response = graph.invoke({
#         "messages": [HumanMessage(content=input_message)]
#     }, {"recursion_limit": 150})
#     return response['messages'][1].content


# inputs = gr.components.Textbox(lines=2, placeholder="Enter your query here...")
# outputs = gr.components.Markdown()

# demo = gr.Interface(
#     fn=run_graph,
#     inputs=inputs,
#     outputs=outputs
# )

# demo.launch()